<a href="https://colab.research.google.com/github/HJoonKwon/ml_fundamentals/blob/main/NaiveBayes_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Naive Bayes Algorithm 

- Supervised learning (We need training data) 
- For classification 
- Based on Bayes' Theorem
- "Naive" means we assume features are independent to each other. 

### How does it work?
- Based on Bayes' theorm, we can calculate posterior probability using prior proabibility, likelihood, and evidence(or marginal probability).
 $$P(A|B) =  \frac{P(B|A)P(A)}{P(B)} $$
- We can just apply the Bayes' theorem to prediction for the probability of the output(classification or regression). Let's assume that the training data has ```n``` number of features, and we want to predict the probability of ```y``` given ```X```. Then,
 $$P(y|X) = P(y|x_1, ..., x_n) = \frac{P(x_1, .., x_n|y)P(y)}{P(x_1, ..., x_n)}  = \frac{P(x_1|y)...P(x_n|y)P(y)}{P(x_1)...P(x_n)}$$ 

 ### What kind of data can it handle?
 - Continuous (Gaussian Naive Bayes)
 - Discrete (Binary/Multinomial Naive Bayes) 

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler

### 1) Prepare for the dataset 
- We are going to use the breast cancer dataset provided by scikit-learn. 
- We can see that all features are continuous, so the Gaussian Naive Bayes would be our choice. 

In [2]:
data = load_breast_cancer() 

In [11]:
print(data['data'].shape)
print(data['feature_names'])
print(data['data'][0])
print(data['target_names'])
print(data['target'])

(569, 30)
['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']
[1.799e+01 1.038e+01 1.228e+02 1.001e+03 1.184e-01 2.776e-01 3.001e-01
 1.471e-01 2.419e-01 7.871e-02 1.095e+00 9.053e-01 8.589e+00 1.534e+02
 6.399e-03 4.904e-02 5.373e-02 1.587e-02 3.003e-02 6.193e-03 2.538e+01
 1.733e+01 1.846e+02 2.019e+03 1.622e-01 6.656e-01 7.119e-01 2.654e-01
 4.601e-01 1.189e-01]
['malignant' 'benign']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 

### 2) Preprocessing the data

In [12]:
scaler = StandardScaler() 
X = data['data']
y = data['target']
X = scaler.fit_transform(X)
train_X, train_y, test_X, test_y = train_test_split(X, y, test_size=0.2, random_state=42)
print(X[0])

[ 1.09706398 -2.07333501  1.26993369  0.9843749   1.56846633  3.28351467
  2.65287398  2.53247522  2.21751501  2.25574689  2.48973393 -0.56526506
  2.83303087  2.48757756 -0.21400165  1.31686157  0.72402616  0.66081994
  1.14875667  0.90708308  1.88668963 -1.35929347  2.30360062  2.00123749
  1.30768627  2.61666502  2.10952635  2.29607613  2.75062224  1.93701461]


### 3) Define helper functions 

In [ ]:
def calc_mean_var():
  return 

def gaussian_log_likelihood():
  return 

### 4) Gaussian Naive Bayes Implementation 

In [ ]:
class GaussianNaiveBayes():
  def __init__(self):
    pass
  
  def calc_prior(self):
    pass

  def calc_posterior(self):
    pass
  
  def fit(self):
    pass 
  
  def predict(self):
    pass 

## References 
- https://medium.com/@rangavamsi5/na%C3%AFve-bayes-algorithm-implementation-from-scratch-in-python-7b2cc39268b9
- https://towardsdatascience.com/implementing-naive-bayes-algorithm-from-scratch-python-c6880cfc9c41